# Classification using pytorch on Iris dataset


In [10]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.optim as optim


In [11]:
dataset = load_iris()
x = dataset.data
y = dataset.target
x

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

## Load and split data


In [12]:
## scale

x_scaled = StandardScaler().fit_transform(x)
# x_scaled
x_train, x_test, y_train, y_test = train_test_split(
    x_scaled, y, test_size=0.2, random_state=42
)

## Convert to tensors


In [13]:
x_train = torch.tensor(x_train)
y_train = torch.tensor(y_train)

x_test = torch.tensor(x_test)
y_test = torch.tensor(y_test)


## Create NN model


In [14]:
class IrisNN(nn.Module):
    def __init__(self, n_in, n_hidden, n_out):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_in, n_hidden), nn.ReLU(), nn.Linear(n_hidden, n_out)
        )

    def forward(self, x):
        return self.net(x)


model = IrisNN(n_in=4, n_hidden=16, n_out=3)

## Set up Loss and optimizer


In [15]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

## Training Loop


In [16]:
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(x_train.float())
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [10/100], Loss: 0.7398
Epoch [20/100], Loss: 0.5190
Epoch [30/100], Loss: 0.3716
Epoch [40/100], Loss: 0.2802
Epoch [50/100], Loss: 0.1951
Epoch [60/100], Loss: 0.1383
Epoch [70/100], Loss: 0.1039
Epoch [80/100], Loss: 0.0846
Epoch [90/100], Loss: 0.0734
Epoch [100/100], Loss: 0.0666


## Evaluation


In [17]:
model.eval()
with torch.no_grad():
    logits = model(x_train.float())  # shape [N, num_classes]
    _, y_pred = torch.max(logits, dim=1)  # predicted class indices, shape [N]


In [18]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
)

y_true = y_train.cpu().numpy()
y_hat = y_pred.cpu().numpy()

acc = accuracy_score(y_true, y_hat)
prec = precision_score(y_true, y_hat, average="macro")
rec = recall_score(y_true, y_hat, average="macro")
f1 = f1_score(y_true, y_hat, average="macro")

print(f"Accuracy : {acc:.3f}")
print(f"Precision: {prec:.3f}")
print(f"Recall   : {rec:.3f}")
print(f"F1-score : {f1:.3f}")

print("\nPer-class metrics:")
print(classification_report(y_true, y_hat))


Accuracy : 0.975
Precision: 0.975
Recall   : 0.975
F1-score : 0.975

Per-class metrics:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       0.97      0.95      0.96        41
           2       0.95      0.97      0.96        39

    accuracy                           0.97       120
   macro avg       0.97      0.98      0.97       120
weighted avg       0.98      0.97      0.98       120

